In [1]:
import sys

sys.path.append('..')

In [2]:
import argparse
import textwrap
from collections import defaultdict
from pathlib import Path

import sklearn.metrics as metrics

from herbarium.pylib import db

In [3]:
DATA = Path('..') / 'data'

DB = DATA / 'angiosperms.sqlite'

## Get test results

In [4]:
results = defaultdict(list)

rows = db.rows_as_dicts(DB, "select * from tests")
for row in rows:
    key = (row["test_set"], row["split_set"])
    results[key].append(row)

## Sort results by split_set and f1

In [5]:
by_split_set = []

for (test_set, split_set), rows in results.items():
    y_true = [r['target'] for r in rows]
    y_pred = [round(r['pred']) for r in rows]

    accuracy = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    prec = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    
    by_split_set.append((split_set, test_set, accuracy, prec, recall, f1))

In [6]:
by_split_set = sorted(by_split_set, key=lambda t: (t[0], -t[-1]))

## Stats per test

In [7]:
print(f'{"split_set":<20} {"test_set":<35} {"acc":^6}   '
      f'{"prec":^6}   {"recall":^6}   {"f1":^6}')

print(f'{"---------":<20} {"--------":<35} ------   '
      '------   ------   ------')

prev = ''
for s in by_split_set:

    if prev and prev != s[0]:
        print()

    print(f'{s[0]:<20} {s[1]:<35} {s[2]:0.4f}   '
          f'{s[3]:0.4f}   {s[4]:0.4f}   {s[5]:0.4f}')
    
    prev = s[0]

split_set            test_set                             acc      prec    recall     f1  
---------            --------                            ------   ------   ------   ------
flowering            b0_flowers_all_orders_2             0.9538   0.9927   0.9547   0.9733
flowering            b0_flowers_all_orders_1             0.9461   0.9927   0.9459   0.9688

flowering_2_orders   b3_flowers_2_orders_unfrozen_2_acc  0.9543   0.9815   0.9710   0.9762
flowering_2_orders   b1_flowers_2_orders_frozen_2_acc    0.9543   0.9968   0.9557   0.9758
flowering_2_orders   b1_flowers_2_orders_unfrozen_4_acc  0.9528   0.9670   0.9847   0.9758
flowering_2_orders   b1_flowers_2_orders_frozen_1_acc    0.9528   0.9984   0.9527   0.9750
flowering_2_orders   b3_flowers_2_orders_frozen_2        0.9499   0.9968   0.9511   0.9734
flowering_2_orders   b3_flowers_2_orders_unfrozen_2      0.9484   0.9952   0.9511   0.9727

fruiting             b7_fruiting_2                       0.8577   0.9529   0.8517   0.89